# Análise exploratório dos dados (EDA).

In [1]:
import pandas as pd
import re

# Importando o arquivo csv de  treino, quantidade linha e colunas e projeção de algumas linha 
train = pd.read_csv('train.csv')
print("Quantidade linhas:%d"%(train.shape[0]))
print("Quantidade linhas: %d"%(train.shape[1]))

train

Quantidade linhas:31962
Quantidade linhas: 3


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


# Pre processamento

In [9]:
def  process_text(dset, column):
    
    # Transforma as frases da coluna em minuscula
    dset[column] = dset[column].str.lower()
    # pega anomalias e substitui por regex
    dset[column] = dset[column].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    
    return dset

train_clean = process_text(train, "tweet")